# Implementation - Factors and operations

In [1]:
import numpy as np

## 1 Factor data structure

In [2]:
class factor:
    def __init__(self, variables = None, distribution = None):
        if (variables is None) or (distribution is None):
            self.__set_data(None, None, None)
        elif (len(variables) == len(distribution.shape)) is False:
            raise Exception('Data is incorrect')
        else:
            self.__set_data(variables, distribution, distribution.shape)
    
    def __set_data(self, variables, distribution, shape):
        self.__variables    = np.array(variables)
        self.__distribution = np.array(distribution)
        self.__shape        = np.array(shape)
    
    # ----------------------- Info --------------------------
    def is_none(self):
        return True if (self.__variables is None) or (self.__distribution is None) else False
        
    # ----------------------- Getters -----------------------
    def get_variables(self):
        return self.__variables
    
    def get_distribution(self):
        return self.__distribution
    
    def get_shape(self):
        return self.__shape

## 2 Factor operations

### 2.1 Factor product

In [3]:
def find_indices(base, target):
    indices = []
    for element in base:
        indices.append(np.where(element==target)[0][0])
    return np.array(indices)

In [4]:
def factor_product(x1, x2):
    # Check if factors are None
    if x1.is_none() or x2.is_none():
        raise Exception('One of the factors is None')
    
    # Recieve data
    x1_variables    = x1.get_variables()
    x1_shape        = x1.get_shape()
    x1_distribution = x1.get_distribution()
    
    x2_shape        = x2.get_shape()
    x2_variables    = x2.get_variables()
    x2_distribution = x2.get_distribution()
    
    # Check if there is at least one common variables
    common_variables = np.intersect1d(x1_variables, x2_variables)
    
    if common_variables.size == 0:
        raise Exception('Factors do not have common variables')
        
    # Check order of common variables
    common_in_x1_indices = find_indices(common_variables, x1_variables)
    common_in_x2_indices = find_indices(common_variables, x2_variables)
    
    if not np.all(x1_shape[common_in_x1_indices] == x2_shape[common_in_x2_indices]):
        raise Exception('Common variables have different order')
    
    # Calculating variables order of factor product
    x1_not_in_x2_variables = np.setdiff1d(x1_variables, x2_variables, assume_unique=True)
    x2_not_in_x1_variables = np.setdiff1d(x2_variables, x1_variables, assume_unique=True)
    
    x3_variables         = np.array(list(x1_not_in_x2_variables) + list(common_variables) + list(x2_not_in_x1_variables))
    x1_like_x3_variables = np.array(list(x1_not_in_x2_variables) + list(common_variables))
    x2_like_x3_variables = np.array(list(common_variables)       + list(x2_not_in_x1_variables))
        
    # Calculating shape of factor product
    x1_in_x3_indices = find_indices(x1_variables, x3_variables)
    x2_in_x3_indices = find_indices(x2_variables, x3_variables)
    
    x3_shape                   = np.zeros(len(x3_variables), dtype=int)
    x3_shape[x1_in_x3_indices] = x1_shape
    x3_shape[x2_in_x3_indices] = x2_shape
    
    # Prepare distribution
    x1_like_x3_indices = find_indices(x1_variables, x1_like_x3_variables)
    x2_like_x3_indices = find_indices(x2_variables, x2_like_x3_variables)
    
    x1_like_x3_distribution = np.moveaxis(x1_distribution, list(range(len(x1_variables))), x1_like_x3_indices)
    x2_like_x3_distribution = np.moveaxis(x2_distribution, list(range(len(x2_variables))), x2_like_x3_indices)
    
    # Calculating factor product itself
    x3_distribution = np.reshape(np.zeros(int(np.prod(x3_shape))), x3_shape)
    
    x2_not_in_x1_in_x3_indices = find_indices(x2_not_in_x1_variables, x3_variables)
    x2_not_in_x1_in_x3_shape   = x3_shape[x2_not_in_x1_in_x3_indices]
    
    x2_not_in_x1_in_x2_like_x3_indices = find_indices(x2_not_in_x1_variables, x2_like_x3_variables)
    
    for i in np.ndindex(tuple(x2_not_in_x1_in_x3_shape)):
        x3_indices = np.array([slice(None)]*len(x3_shape))
        x3_indices[x2_not_in_x1_in_x3_indices] = i
        
        x2_indices = np.array([slice(None)]*len(x2_shape))
        x2_indices[x2_not_in_x1_in_x2_like_x3_indices] = i
        
        x3_distribution[tuple(x3_indices)] = x1_like_x3_distribution * x2_like_x3_distribution[tuple(x2_indices)]
    
    # Create factor
    x3 = factor(x3_variables, x3_distribution)
    
    return x3

#### Example 1: simple factor product

In [5]:
f1 = np.array([[0.,0.], [0.,0.], [0.,0.]])

f1[0,0] = 0.5
f1[0,1] = 0.8
f1[1,0] = 0.1
f1[1,1] = 0.0
f1[2,0] = 0.3
f1[2,1] = 0.9

phi_1 = factor(['a', 'b'], f1)

In [6]:
f2 = np.array([[0.,0.], [0.,0.]])

f2[0,0] = 0.5
f2[0,1] = 0.7
f2[1,0] = 0.1
f2[1,1] = 0.2

phi_2 = factor(['b', 'c'], f2)

In [7]:
phi_3 = factor_product(phi_1, phi_2)

In [8]:
phi_3.get_variables()

array(['a', 'b', 'c'], dtype='<U1')

#### Example 2: computing joint distribution

### 2.2 Factor marginalization

In [9]:
def factor_marginalization(x, variables):
    variables = np.array(variables)
    
    if x.is_none():
        raise Exception('Factor is None')
    
    if not np.all(np.in1d(variables, x.get_variables())):
        raise Exception('Factor do not contain given variables')
    
    res_variables    = np.setdiff1d(x.get_variables(), variables, assume_unique=True)
    res_distribution = np.sum(x.get_distribution(),
                              tuple(np.where(np.isin(x.get_variables(), variables))[0]))
    
    return factor(res_variables, res_distribution)

#### Example

In [10]:
phi_4 = factor_marginalization(phi_3, ['b'])

In [11]:
phi_4.get_variables()

array(['a', 'c'], dtype='<U1')

### 2.3 Factor reduction

In [12]:
def factor_reduction(x, variable, value):
    if x.is_none() or (variable is None) or (value is None):
        raise Exception('Input is None')
    
    if not np.any(variable == x.get_variables()):
        raise Exception('Factor do not contain given variable')
    
    if value >= x.get_shape()[np.where(variable==x.get_variables())[0]]:
        raise Exception('Incorrect value of given variable')
    
    res_variables    = np.setdiff1d(x.get_variables(), variable, assume_unique=True)
    res_distribution = np.take(x.get_distribution(),
                               value,
                               int(np.where(variable==x.get_variables())[0]))
    
    return factor(res_variables, res_distribution)

#### Example

In [13]:
factor_reduction(phi_3, 'c', 0).get_variables()

array(['a', 'b'], dtype='<U1')

### 2.4 Joint distribution

In [14]:
def joint_distribution(ar):
    for element in ar:
        if element.is_none():
            raise Exception('Factor is None')
    
    res = ar[0]
    for element in ar[1:]:
        res = factor_product(res, element)
    
    return res

#### Example

In [15]:
joint_distribution([phi_1, phi_2]).get_variables()

array(['a', 'b', 'c'], dtype='<U1')